In [52]:
import pandas as pd
from math import sqrt
import numpy as np

In [53]:
movies_data = pd.read_csv('movies.csv')
ratings_data = pd.read_csv('ratings.csv')

In [54]:
print(movies_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None


In [55]:
print(ratings_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534942 entries, 0 to 534941
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     534942 non-null  int64  
 1   movieId    534942 non-null  int64  
 2   rating     534942 non-null  float64
 3   timestamp  534942 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 16.3 MB
None


In [70]:
# User input for movies_data DataFrame
userInput_movies_data = [
    {'movieId': 1, 'title': 'Breakfast Club, The', 'genres': 'Comedy|Drama'},
    {'movieId': 2, 'title': 'Toy Story', 'genres': 'Animation|Adventure|Comedy'},
    {'movieId': 3, 'title': 'Jumanji', 'genres': 'Adventure|Comedy|Fantasy'},
    {'movieId': 4, 'title': 'Pulp Fiction', 'genres': 'Crime|Drama'},
    {'movieId': 5, 'title': 'Akira', 'genres': 'Animation|Sci-Fi'}
]

inputMovies_movies_data = pd.DataFrame(userInput_movies_data)
print(inputMovies_movies_data)

   movieId                title                      genres
0        1  Breakfast Club, The                Comedy|Drama
1        2            Toy Story  Animation|Adventure|Comedy
2        3              Jumanji    Adventure|Comedy|Fantasy
3        4         Pulp Fiction                 Crime|Drama
4        5                Akira            Animation|Sci-Fi


In [71]:
# User input for ratings_data DataFrame
userInput_ratings_data = [
    {'userId': 1, 'movieId': 1, 'rating': 5, 'timestamp': 1638255543},
    {'userId': 1, 'movieId': 2, 'rating': 1, 'timestamp': 1638255543},
    {'userId': 1, 'movieId': 3, 'rating': 1, 'timestamp': 1638255543},
    {'userId': 1, 'movieId': 4, 'rating': 5, 'timestamp': 1638255543},
    {'userId': 1, 'movieId': 5, 'rating': 4.5, 'timestamp': 1638255543}
]

inputMovies_ratings_data = pd.DataFrame(userInput_ratings_data)
print(inputMovies_ratings_data)


   userId  movieId  rating   timestamp
0       1        1     5.0  1638255543
1       1        2     1.0  1638255543
2       1        3     1.0  1638255543
3       1        4     5.0  1638255543
4       1        5     4.5  1638255543


In [75]:
# Merge the two DataFrames based on 'movieId'
inputMovies = pd.merge(inputMovies_ratings_data, inputMovies_movies_data, on='movieId')

# Select desired columns
inputMovies = inputMovies[['movieId', 'title', 'rating', 'genres', 'userId', 'timestamp']]

# Display the merged DataFrame
print(inputMovies)


   movieId                title  rating                      genres  userId  \
0        1  Breakfast Club, The     5.0                Comedy|Drama       1   
1        2            Toy Story     1.0  Animation|Adventure|Comedy       1   
2        3              Jumanji     1.0    Adventure|Comedy|Fantasy       1   
3        4         Pulp Fiction     5.0                 Crime|Drama       1   
4        5                Akira     4.5            Animation|Sci-Fi       1   

    timestamp  
0  1638255543  
1  1638255543  
2  1638255543  
3  1638255543  
4  1638255543  


In [82]:
# Convert titles to lowercase for case-insensitive matching
movies_data['title_lower'] = movies_data['title'].str.lower()
inputMovies['title_lower'] = inputMovies['title'].str.lower()

# Merge based on lowercase titles
inputMovies = pd.merge(movies_data, inputMovies, left_on='title_lower', right_on='title_lower', how='inner')

# Print columns after the merge
print("Columns after merge:", inputMovies.columns)

# Select desired columns
inputMovies = inputMovies[['movieId', 'title', 'rating','genres','userid']]

# Rename the columns for clarity
inputMovies.columns = ['movieId', 'title', 'rating','genres','userid']

print(inputMovies)


KeyError: ignored

In [59]:
userSubset = ratings_data[ratings_data['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.groupby('movieId').count())

Empty DataFrame
Columns: [userId, rating, timestamp]
Index: []


In [60]:
# Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

def take_5_elem(x):
    # print (len(x[1]))
    return len(x[1])


#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[]


<ipython-input-60-733081cb0516>:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)


In [80]:
inputId = movies_data[movies_data['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
#inputMovies = inputMovies.drop('year', 1) #we don't really need this at the moment
inputMovies = inputMovies[['movieId','title','rating']]
print(inputMovies)

Empty DataFrame
Columns: [movieId, title, rating]
Index: []


In [62]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

ValueError: ignored

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

In [ ]:
topUsersRating=topUsers.merge(ratings_data, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head(100))

In [ ]:
#Multiplies the similarity by the user’s ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

In [ ]:
#Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

In [ ]:
#Creates an empty dataframe
recommendation_data = pd.DataFrame()

#Now we take the weighted average
recommendation_data['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_data['movieId'] = tempTopUsersRating.index
print(recommendation_data.head(10))

In [ ]:
recommendation_data = recommendation_data.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_data)

In [ ]:
recommended_movie=movies_df.loc[movies_df['movieId'].isin(recommendation_df['movieId'])]

#we don't want to recommend the same movie
recommended_movie=recommended_movie.loc[~recommended_movie.movieId.isin(userSubset['movieId'])]

print(recommended_movie)